In [5]:
names=open('names.txt','r').read().splitlines()

In [6]:
names

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn',
 'abigail',
 'emily',
 'elizabeth',
 'mila',
 'ella',
 'avery',
 'sofia',
 'camila',
 'aria',
 'scarlett',
 'victoria',
 'madison',
 'luna',
 'grace',
 'chloe',
 'penelope',
 'layla',
 'riley',
 'zoey',
 'nora',
 'lily',
 'eleanor',
 'hannah',
 'lillian',
 'addison',
 'aubrey',
 'ellie',
 'stella',
 'natalie',
 'zoe',
 'leah',
 'hazel',
 'violet',
 'aurora',
 'savannah',
 'audrey',
 'brooklyn',
 'bella',
 'claire',
 'skylar',
 'lucy',
 'paisley',
 'everly',
 'anna',
 'caroline',
 'nova',
 'genesis',
 'emilia',
 'kennedy',
 'samantha',
 'maya',
 'willow',
 'kinsley',
 'naomi',
 'aaliyah',
 'elena',
 'sarah',
 'ariana',
 'allison',
 'gabriella',
 'alice',
 'madelyn',
 'cora',
 'ruby',
 'eva',
 'serenity',
 'autumn',
 'adeline',
 'hailey',
 'gianna',
 'valentina',
 'isla',
 'eliana',
 'quinn',
 'nevaeh',
 'ivy',
 'sadie',
 'piper',
 'lydia',
 'alexa',
 'josephine',
 'emery',
 'julia'

In [7]:
names[0]

'emma'

Building a basic bigram model
This counts all the bigrams of the characters and based on these predicts the new characters

The below code creates all the bigrams zip basically makes an iterator. we have added start and end as characters as well


In [8]:
b={}
for w in names:
    chs=['<S>']+list(w)+['<E>']
    for ch1,ch2 in zip(chs,chs[1:]):
        bigram=(ch1,ch2)
        b[bigram]=b.get(bigram,0)+1   #here b.bigram is actually b[bigram] but 0 in case it isn't there
        

In [9]:
len(b)

627

We are gonna make a 2D array of all these character bigrams with the rows and columns numbers being the characters a-z with start and end char at last from 1-28 

In [10]:
import torch
N=torch.zeros((27,27),dtype=torch.int32)

In [11]:
chars=sorted(list(set(''.join(names))))  #since sets doesn't allow duplicates, so it gives us all the letters and then we get the sorted list

In [12]:
#we basically replaced the end and start characters with . character and 

In [13]:
stoi={s:i+1 for i,s in enumerate(chars)} #this is to map integers to the characters
stoi['.']=0
stoi
itos={i:s for s,i in stoi.items()}

In [14]:
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [15]:
for w in names:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]  
        ix2=stoi[ch2]   #ix1 and ix2 give the index of the element where the two characters make a bigram
        N[ix1,ix2]+=1

In [16]:
len(N[0,:])
#so basically the first row gives us how many times .,a,b,c,..... follow '.'

27

In [17]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [18]:
p=N[0].float()
p=p/p.sum()  # with the sum we are getting the probablities of each index(which in itself is a word) being the first word
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [19]:
g=torch.Generator().manual_seed(2147483647)  #this is to get a deterministic value always
idx=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item() 
#how the torch.multinomial works is that it samples(the index) the num_samples we have given it based on their probabilities
# for instances if the first element has 0.5 probablity, 2nd has 0.25 probablity, 3rd has 0.25 probability
# and we sample 4 elements index 0 would be 2 and the rest would be one each
#so here we get the most probable element for the first character
# .item() gives us the item,without .item()we would have gotten tensor([13]) wrapped
idx

10

In [20]:
g=torch.Generator().manual_seed(2147483647)
ix=0
##this code is for sampling from the model
for i in range(10):
    out=[]
    while True:
        p=N[ix].float()
        p=p/p.sum()
        ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:                  #final character or the end character
            break

    print(''.join(out))
#so what we are doing here sampling the first word
#then keeping that as a first word and sampling the second word
#we are doing this till we get . at the end


junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.


In [21]:
#while above we are first getting p(row) from N and then normalizing it we can directly normalize the whole 2D array and then get p
P=(N+1).float() #the +1 is for smoothening since some of the counts will be zero which can give us infinity in the negative log likelihood
P=P/(P.sum(1,keepdim=True))
#the keepdim=True doesn't let the .sum squeeze the dimension if we did P.sum(1) we would have gotten an array of length (27)
#with keepdim=True we get (27,1) and then when it is broadcasted we get (27,27), if (27) would be broadcasted it would first become (1,27) and then (27,27)


In [22]:
P[0].sum()

tensor(1.)

In [23]:
sum_negative_probs=0
n=0
for w in names:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]  
        ix2=stoi[ch2]
        prob=P[ix1,ix2] 
        #this will give us the probablities of all of these bigrams, 
        #since these are in the training set and we have to maximize them,
        #so the log probabilities would should also be higher and the negative avg log probs have to be less
        nll=-torch.log(prob)
        n+=1
        sum_negative_probs+=nll 
sum_negative_probs/=n 
sum_negative_probs.item()

2.4543561935424805

In [24]:
names

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn',
 'abigail',
 'emily',
 'elizabeth',
 'mila',
 'ella',
 'avery',
 'sofia',
 'camila',
 'aria',
 'scarlett',
 'victoria',
 'madison',
 'luna',
 'grace',
 'chloe',
 'penelope',
 'layla',
 'riley',
 'zoey',
 'nora',
 'lily',
 'eleanor',
 'hannah',
 'lillian',
 'addison',
 'aubrey',
 'ellie',
 'stella',
 'natalie',
 'zoe',
 'leah',
 'hazel',
 'violet',
 'aurora',
 'savannah',
 'audrey',
 'brooklyn',
 'bella',
 'claire',
 'skylar',
 'lucy',
 'paisley',
 'everly',
 'anna',
 'caroline',
 'nova',
 'genesis',
 'emilia',
 'kennedy',
 'samantha',
 'maya',
 'willow',
 'kinsley',
 'naomi',
 'aaliyah',
 'elena',
 'sarah',
 'ariana',
 'allison',
 'gabriella',
 'alice',
 'madelyn',
 'cora',
 'ruby',
 'eva',
 'serenity',
 'autumn',
 'adeline',
 'hailey',
 'gianna',
 'valentina',
 'isla',
 'eliana',
 'quinn',
 'nevaeh',
 'ivy',
 'sadie',
 'piper',
 'lydia',
 'alexa',
 'josephine',
 'emery',
 'julia'

In [25]:
##now we are gonna use a simple neural networks for the above problem
##for which we first get the data ready
xs=[]
ys=[]
for w in names[:1]:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        xs.append(ix1) #since we are gonna still use bigram for the neural network,xs is the input which is the first
        #character and then ix2 which is the 2nd character is the output
        ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [26]:
xs

tensor([ 0,  5, 13, 13,  1])

In [27]:
## using one hot encoding first
from torch.nn.functional import one_hot
x_enc=one_hot(xs,27).float()
y_enc=one_hot(ys,27).float()

x_enc.shape

torch.Size([5, 27])

In [28]:
##shape of the input is (5,27), so basically every input element is a (1,27) vector and we have to project
##each one further to a (1,27) dim vector

In [29]:
w=torch.randn((27,27),requires_grad=True)
##randomly initializing w

In [30]:
logits=x_enc @ w ##this gives us (5,27) matrix
logits
counts=logits.exp() ##we will exponentiate them to get counts 
counts

tensor([[ 0.6502,  2.3452,  0.7711,  0.6670,  0.8489,  1.6483,  2.1101,  1.4735,
          0.6770,  0.4009,  6.1305,  0.7481,  0.9331,  0.9026,  3.5680,  0.9984,
          0.5065,  1.1151,  2.5365,  3.7957,  0.2374,  0.2491,  0.8187,  0.2346,
          0.4846,  1.4227,  1.3483],
        [ 1.7577,  3.4292,  0.3909,  1.9623,  6.3053,  0.6653,  2.1090,  0.5481,
          1.2610,  1.8200,  0.2554,  0.2927,  0.9439,  0.4583,  0.4527,  0.2123,
          0.5939,  2.0323,  1.5772,  1.5001,  2.7580,  0.6509,  5.3033,  4.3768,
          0.8151,  0.2314,  2.8280],
        [ 1.0957,  0.4670,  1.3423,  0.2873,  2.6528,  1.9253,  0.7633,  1.7612,
          0.2849,  2.0856,  1.7331,  0.2278,  5.1841,  0.8629,  0.9940,  0.4497,
          0.3348,  4.2047,  2.7191,  1.2819,  1.2273,  0.3503,  0.8176, 16.4487,
          0.8482,  0.2083,  7.9100],
        [ 1.0957,  0.4670,  1.3423,  0.2873,  2.6528,  1.9253,  0.7633,  1.7612,
          0.2849,  2.0856,  1.7331,  0.2278,  5.1841,  0.8629,  0.9940,  0.4497

In [31]:
prob=(counts)/(counts.sum(dim=1,keepdim=True)) ##this gives us the probabilities
prob

tensor([[0.0173, 0.0623, 0.0205, 0.0177, 0.0226, 0.0438, 0.0561, 0.0392, 0.0180,
         0.0107, 0.1630, 0.0199, 0.0248, 0.0240, 0.0948, 0.0265, 0.0135, 0.0296,
         0.0674, 0.1009, 0.0063, 0.0066, 0.0218, 0.0062, 0.0129, 0.0378, 0.0358],
        [0.0386, 0.0753, 0.0086, 0.0431, 0.1385, 0.0146, 0.0463, 0.0120, 0.0277,
         0.0400, 0.0056, 0.0064, 0.0207, 0.0101, 0.0099, 0.0047, 0.0130, 0.0446,
         0.0346, 0.0329, 0.0606, 0.0143, 0.1165, 0.0961, 0.0179, 0.0051, 0.0621],
        [0.0187, 0.0080, 0.0230, 0.0049, 0.0454, 0.0329, 0.0131, 0.0301, 0.0049,
         0.0357, 0.0296, 0.0039, 0.0887, 0.0148, 0.0170, 0.0077, 0.0057, 0.0719,
         0.0465, 0.0219, 0.0210, 0.0060, 0.0140, 0.2813, 0.0145, 0.0036, 0.1353],
        [0.0187, 0.0080, 0.0230, 0.0049, 0.0454, 0.0329, 0.0131, 0.0301, 0.0049,
         0.0357, 0.0296, 0.0039, 0.0887, 0.0148, 0.0170, 0.0077, 0.0057, 0.0719,
         0.0465, 0.0219, 0.0210, 0.0060, 0.0140, 0.2813, 0.0145, 0.0036, 0.1353],
        [0.0289, 0.0163,

In [32]:
##basically what the prob gives us is like for the first row, it gives us the probabilities of all these indexes
##being the next character
nlls=torch.zeros(5)
for i in range(5):
    x_i=xs[i].item()
    y_i=ys[i].item()
    P=prob[i,y_i] ##probability of that character
    ##for instance if the next character is e after . so prob[x_i][y_i] gives us its probability
    log_prob=torch.log(P)
    nlls[i]=-(log_prob)
nlls
loss=nlls.mean()

In [33]:
loss

tensor(4.0636, grad_fn=<MeanBackward0>)

Complete forward and backward propagation

In [43]:
xs=[]
ys=[]
for w in names:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        xs.append(ix1) #since we are gonna still use bigram for the neural network,xs is the input which is the first
        #character and then ix2 which is the 2nd character is the output
        ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [44]:
## using one hot encoding first
from torch.nn.functional import one_hot
x_enc=one_hot(xs,27).float()
y_enc=one_hot(ys,27).float()

x_enc.shape

torch.Size([228146, 27])

In [45]:
w=torch.randn((27,27),requires_grad=True)
##randomly initializing w and keeping requires_grad=True since we have to calculate loss and then gradients

In [46]:
num_element=xs.nelement()
num_element

228146

In [47]:
##basically what the prob gives us is like for the first row, it gives us the probabilities of all these indexes
##being the next character
for i in range(1000):
    nlls=torch.zeros(num_element)
    logits=x_enc @ w ##outputs of the neural network, predicts log counts
    counts=logits.exp()
    prob=counts/counts.sum(1,keepdim=True)
    loss=-prob[torch.arange(num_element),ys].log().mean() ##here we are calculating the probabilities for the characters
    ##that actually follow the current character, that is for those in the current character
    
    w.grad=None
    loss.backward()
    w.data+=(-50*w.grad)
    print(loss.item())
    ##probability of that character
    ##for instance if the next character is e after . so prob[x_i][y_i] gives us its probability
    

3.868350028991699
3.4535961151123047
3.2144572734832764
3.0617480278015137
2.9559857845306396
2.879060983657837
2.821563243865967
2.777290105819702
2.742220163345337
2.713712453842163
2.6899969577789307
2.6698853969573975
2.6525774002075195
2.6375186443328857
2.6243085861206055
2.612644910812378
2.6022934913635254
2.593065023422241
2.5848042964935303
2.577382802963257
2.5706918239593506
2.5646398067474365
2.559147596359253
2.5541481971740723
2.5495824813842773
2.545401096343994
2.5415592193603516
2.5380189418792725
2.53474760055542
2.5317163467407227
2.5288987159729004
2.526273250579834
2.523820161819458
2.5215227603912354
2.5193655490875244
2.517335891723633
2.5154221057891846
2.51361346244812
2.51190185546875
2.5102789402008057
2.508737564086914
2.5072720050811768
2.505876064300537
2.5045454502105713
2.503275156021118
2.502061128616333
2.5009005069732666
2.499788999557495
2.4987242221832275
2.4977028369903564
2.49672269821167
2.495781660079956
2.4948770999908447
2.4940073490142822
2.

In [48]:
w

tensor([[-4.6455e+00,  2.2750e+00,  1.0580e+00,  1.2242e+00,  1.3158e+00,
          1.2170e+00, -8.3893e-02,  3.8896e-01,  6.5631e-01,  2.6496e-01,
          1.6757e+00,  1.8773e+00,  1.2434e+00,  1.7225e+00,  9.2731e-01,
         -1.4065e-01,  1.2726e-01, -1.5967e+00,  1.2852e+00,  1.5114e+00,
          1.0595e+00, -1.7621e+00, -1.8743e-01, -3.9028e-01, -1.2200e+00,
          1.6538e-01,  7.1735e-01],
        [ 2.3341e+00, -1.4601e-01, -1.7336e-01, -3.1405e-01,  4.8212e-01,
          7.2805e-02, -1.5689e+00, -1.3428e+00,  1.2877e+00,  9.4175e-01,
         -1.3020e+00, -1.2466e-01,  1.3684e+00,  9.3201e-01,  2.1344e+00,
         -2.3236e+00, -2.0601e+00, -2.3724e+00,  1.6239e+00,  5.5252e-01,
          6.5553e-02, -5.2398e-01,  2.5945e-01, -1.3854e+00, -1.2628e+00,
          1.1588e+00, -3.9143e-01],
        [ 2.1225e+00,  3.1702e+00,  9.7616e-01, -1.5193e+00,  1.5446e+00,
          3.8867e+00, -1.9376e+00, -1.7817e+00,  1.0576e+00,  2.7754e+00,
         -1.4885e+00, -1.7725e+00,  2.01

In [56]:
##now to sample
import torch.nn.functional as F 
g=torch.Generator().manual_seed(2147483647)

for i in range(10):
    ix=0
    out=[]
    while True:
        x_enc=F.one_hot(torch.tensor(ix),num_classes=27).float()
        logits=x_enc @ w ##outputs of the neural network, predicts log counts
        counts=logits.exp()
        prob=counts/counts.sum()
        ix=torch.multinomial(prob,num_samples=1,replacement=True,generator=g).item()
        out.append(itos[ix])
        if ix==0:                  #final character or the end character
            break

    print(''.join(out))
    
    

junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.
